In [1]:
import pandas as pd

# !pip install pandasql
import pandasql as ps
import math


# Analysis and data transformation for all the files 

In [3]:
# url = [
#     "https://raw.githubusercontent.com/BigTimeStats/beach-volleyball/master/data/full_archive/full_archive.csv"
# ]

big_stat_df_with_nan = pd.read_csv(
    "./data/BigTimeStats.csv",
    index_col=None,
    header=0,
    encoding="latin-1",
    usecols=range(33),
)
big_stat_df_with_nan.shape

(85509, 33)

In [4]:
big_stat_df = big_stat_df_with_nan.dropna(axis=0, thresh=5)


In [5]:
name = "Grigorii Voleanin"
dob = "1967-04-01"
hgt = 77.0


big_stat_df.loc[big_stat_df["l_player1"] == name, "l_p1_hgt"] = hgt
big_stat_df.loc[big_stat_df["l_player2"] == name, "l_p2_hgt"] = hgt

big_stat_df.loc[big_stat_df["l_player1"] == name, "l_p1_birthdate"] = dob
big_stat_df.loc[big_stat_df["l_player2"] == name, "l_p2_birthdate"] = dob

In [6]:
big_stat_df[big_stat_df["l_player2"] == name][
    ["l_player2", "l_p2_hgt", "l_p2_birthdate"]
]

,l_player2,l_p2_hgt,l_p2_birthdate
38825,Grigorii Voleanin,77.0,1967-04-01
40438,Grigorii Voleanin,77.0,1967-04-01
44181,Grigorii Voleanin,77.0,1967-04-01
49342,Grigorii Voleanin,77.0,1967-04-01
53238,Grigorii Voleanin,77.0,1967-04-01


In [7]:
date_columns = ["w_p1_birthdate", "w_p2_birthdate", "l_p1_birthdate", "l_p2_birthdate"]
for col in date_columns:
    big_stat_df[col] = pd.to_datetime(big_stat_df[col], errors="coerce")
    big_stat_df[col] = big_stat_df[col].apply(
        lambda x: x.replace(year=x.year - 100) if x.year > 2015 else x
    )
    big_stat_df["date"] = pd.to_datetime(big_stat_df["date"], errors="coerce")


In [8]:
##reading teamblack
tb_df_cw = pd.read_excel("./data/TeamBlackData.xlsx", sheet_name="Challenge Women")
tb_df_cw["gender"] = "F"  # 0 for female

tb_df_cm = pd.read_excel("./data/TeamBlackData.xlsx", sheet_name="Challenge Men")
tb_df_cm["gender"] = "M"  # 1 for male

tb_df_elite16w = pd.read_excel("./data/TeamBlackData.xlsx", sheet_name="Elite 16 Women")
tb_df_elite16w["gender"] = "F"

tb_df_elite16m = pd.read_excel("./data/TeamBlackData.xlsx", sheet_name="Elite 16 Men")
tb_df_elite16m["gender"] = "M"

tb_df = pd.concat(
    [tb_df_cw, tb_df_cm, tb_df_elite16w, tb_df_elite16m], ignore_index=True
)
tb_df["isBlack_Challenge"] = tb_df["TypeOfEvent"] == "Challenge"
tb_df["isBlack_Elite16"] = tb_df["TypeOfEvent"] == "Elite 16"

tb_df.to_csv("./data/out/full_team_black.csv")


In [9]:
# Keep only unique rows based on the 'FIVB_ID' column
tb_df_unique = tb_df.drop_duplicates(subset="FIVB_ID", keep="first")
tb_df_unique.shape

(130, 15)

we have only 130 unoque athletes from teamBlack

In [10]:
## get all scraped data - will be based on this and update it with data from
allplayers_df = pd.read_csv("./data/allplayers.csv")
# transform height
# allplayers_df.describe(include="all")

##### Transform Height to number in inches and in cm 

In [90]:
def convert_to_inches(height_str):
    if isinstance(height_str, str):
        feet, inches = map(int, height_str.strip('"').split("'"))
        # print(feet, inches)
        total_inches = (feet * 12) + inches
        return total_inches
    else:
        return math.nan


# Function to convert height from inches to centimeters
def convert_to_cm(height_inches):
    return height_inches * 2.54


# 9818 records with height


In [91]:
allplayers_df["height_inches"] = allplayers_df["height_in"].apply(convert_to_inches)
allplayers_df["height_cm"] = allplayers_df["height_inches"].apply(convert_to_cm)
allplayers_df = allplayers_df.drop("height_in", axis=1)
def parse_date(date_str):
    try:
        if pd.isna(date_str):
            return None
        # Split the date string to extract the date part (e.g., "April 28, 1973")
        date_part = date_str.split(" (")[0]

        # # Convert the date part to a datetime object
        date_object = pd.to_datetime(date_part, format="%B %d, %Y")

        return date_object
    except ValueError:
        return None


# Test the function with the example date string
# date_string = "April 28"
# parsed_date = parse_date(date_string)

# print(parsed_date)

allplayers_df["DOB_str"] = allplayers_df["DOB"]
allplayers_df["DOB"] = allplayers_df["DOB_str"].apply(parse_date)

In [92]:
allplayers_df[["height_in", "height_inches", "height_cm"]].describe(include="all")


,height_in,height_inches,height_cm
count,9818,9818.000000,9818.000000
unique,31,NaN,NaN
top,"6'0""",NaN,NaN
freq,1027,NaN,NaN
mean,NaN,72.546038,184.266936
std,NaN,3.761149,9.553320
min,NaN,54.000000,137.160000
25%,NaN,70.000000,177.800000
50%,NaN,72.000000,182.880000
75%,NaN,75.000000,190.500000


#####  Enrich  data of athletes height from big stats 

In [328]:
matches_df = big_stat_df[
    [
        "circuit",
        "tournament",
        "country",
        "year",
        "date",
        "gender",
        "match_num",
        "score",
        "duration",
        "bracket",
        "round",
        "w_player1",
        "w_p1_birthdate",
        "w_player2",
        "w_p2_birthdate",
        "l_player1",
        "l_p1_birthdate",
        "l_p2_birthdate",
    ]
]
# matches_df.head()

In [498]:
def extract_scores_and_rounds(score):
    try:
        if pd.isna(score):
            return math.nan, math.nan, math.nan

        rounds = score.count(",") + 1
        score_parts = score.split(",")
        total_win_score = 0
        total_lost_score = 0

        for score_part in score_parts:
            win_score, lost_score = score_part.strip().split("-")
            total_win_score += int(win_score)
            total_lost_score += int(lost_score)

        return total_win_score, total_lost_score, rounds
    except ValueError:
        return math.nan, math.nan, math.nan


matches_df[["win_score", "lost_score", "num_rounds"]] = (
    matches_df["score"].apply(extract_scores_and_rounds).apply(pd.Series)
)
# matches_df[["win_score", "lost_score", "num_rounds", "score"]].describe(include="all")


C:\Users\Marina\AppData\Local\Temp\ipykernel_13004\2814655153.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_df[["win_score", "lost_score", "num_rounds"]] = (


In [499]:
matches_df["match_id"] = matches_df.index + 1


C:\Users\Marina\AppData\Local\Temp\ipykernel_13004\36878012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_df["match_id"] = matches_df.index + 1


In [450]:
## Connecting by using player name

# big_stat_df.describe()

w_p1_dict = {
    "gender": "gender",
    "w_player1": "player_name",
    "w_p1_birthdate": "DOB",
    "w_p1_hgt": "height_inches",
    "w_p1_country": "player_country",
    "duration": "duration",
    "w_player2": "partner",
}
# w_p1_dict.keys()
w_p1_df = big_stat_df[w_p1_dict.keys()].rename(columns=w_p1_dict)

w_p2_dict = {
    "gender": "gender",
    "w_player2": "player_name",
    "w_p2_birthdate": "DOB",
    "w_p2_hgt": "height_inches",
    "w_p2_country": "player_country",
    "duration": "duration",
    "w_player1": "partner",
}

w_p2_df = big_stat_df[w_p2_dict.keys()].rename(columns=w_p2_dict)

l_p1_dict = {
    "gender": "gender",
    "l_player1": "player_name",
    "l_p1_birthdate": "DOB",
    "l_p1_hgt": "height_inches",
    "l_p1_country": "player_country",
    "duration": "duration",
    "l_player2": "partner",
}

l_p1_df = big_stat_df[l_p1_dict.keys()].rename(columns=l_p1_dict)

l_p2_dict = {
    "gender": "gender",
    "l_player2": "player_name",
    "l_p2_birthdate": "DOB",
    "l_p2_hgt": "height_inches",
    "l_p2_country": "player_country",
    "duration": "duration",
    "l_player1": "partner",
}
l_p2_df = big_stat_df[l_p2_dict.keys()].rename(columns=l_p2_dict)

In [451]:
big_stat_players_df = pd.concat([w_p1_df, w_p2_df, l_p1_df, l_p2_df])
big_stat_players_df.shape

(342036, 7)

In [334]:
query = """
    SELECT BVB_ID, FIVB_Name
    FROM tb_df_unique
    WHERE tb_df_unique.FIVB_Name not in (
        SELECT player_name
        FROM big_stat_players_df )
"""

# Execute the SQL query
not_merged_players_df = ps.sqldf(query, globals())

# Display the players who didn't merge into allplayers_df
print(not_merged_players_df)

    BVB_ID                           FIVB_Name
0     5887            Carolina Solberg Salgado
1     8022           Barbara Seixas De Freitas
2     9593                Elize Secomandi Maia
3    18464             Thamela Coradello Galil
4    14458         Andressa Cavalcanti Ramalho
5    16883          Vitoria De Souza Rodrigues
6     8395           Mariafe Artacho Del Solar
7    17054                    Mexime Van Driel
8     7914                Taru Lahti-Liukkonen
9    14460                    Taina Silva Bigi
10   15283        Victoria Lopes Pereira Tosta
11   18426  Vicente Orlando Droguett Torrealba
12   14454        George Souto Maior Wanderley
13   15355                  Andre Loyola Stein
14    8410                  Christopher Mchugh
15   13275                 Hendrik Nikolai Mol
16   18352                       Andrew Benesh
17   15323          Arthur Diego Mariano Lanci
18    8338    Adrian Ignacio Carambula Raurich
19    2077               Pablo Herrera Allepuz
20    7783   

In [335]:
# Remove duplicates from the DataFrame
big_stat_players_df.drop_duplicates(subset="player_name", inplace=True)
big_stat_players_df.reset_index(drop=True, inplace=True)

print(big_stat_players_df.player_name.describe())


count          10076
unique         10076
top       Kevin Wong
freq               1
Name: player_name, dtype: object


In [336]:
big_stat_players_df[big_stat_players_df.duplicated(subset="player_name", keep=False)]


,gender,player_name,DOB,height_inches,player_country


In [285]:
## find ids for the players by name and birthdate , assign the ids
## if there is no birthdate, check for player and height
## else just first occurance of the name

# big_stat_players_df vs allplayers_df

# allplayers_df.describe(include="all")  # player_name DOB player_id

# transform allplayers_df.DOB


### Merge scraped file and file from match statistics 

In [337]:
## MErge
from fuzzywuzzy import fuzz


def find_name(string1, names, threshold=85):
    for name in names:
        words1 = string1.lower().split()
        words2 = name.lower().split()  ## big stat names
        count = 0
        for word1 in words1:
            for word2 in words2:
                similarity_score = fuzz.token_sort_ratio(word1, word2)
                if similarity_score >= threshold:
                    count = count + 1
                    break
        if (count >= 2) and (len(words2) == count):
            return name
    return None


# # Test
# string1 = "Andressa Cavalcanti Ramalho"
# result1 = find_name(string1, big_stat_players_df["player_name"], threshold=85)
# print(result1)  # Output: True


# # # # Create a copy of allplayers_df with a new column 'best_match_player_name'
allplayers_copy = allplayers_df.copy()
allplayers_copy["big_stat_player_name"] = allplayers_copy["player_name"].apply(
    lambda x: find_name(x, big_stat_players_df["player_name"], threshold=85)
)

# allplayers_copy.to_csv("./data/out/allplayers_updated.csv", index=False)

In [500]:
allplayers_copy[["player_name", "big_stat_player_name"]][
    allplayers_copy.big_stat_player_name.isna()
]

,player_name,big_stat_player_name
1,Rifat Agi,None
2,Todd Ahmadi,None
7,Robert Bell,None
8,Jeff Bellandi,None
9,Jud Buechler,None
...,...,...
21765,Jinyang Song,None
21766,Abudula Dilinaer,None
21767,Jindi Dong,None
21768,Jiayi Xing,None


In [341]:
allplayers_copy.columns

Index(['player_id', 'player_name', 'player_country', 'DOB', 'home_town',
       'resides', 'total_first', 'total_second', 'total_third', 'total_forth',
       'total_played', 'total_points', 'podium%', 'height_inches', 'height_cm',
       'DOB_date', 'DOB_str', 'big_stat_player_name'],
      dtype='object')

In [501]:
allplayers_copy.loc[
    allplayers_copy["player_id"] == 7914, "big_stat_player_name"
] = "Taru Lahti"


In [502]:
allplayers_copy[allplayers_copy["player_id"] == 7914][
    ["player_id", "player_name", "big_stat_player_name", "DOB"]
]


,player_id,player_name,big_stat_player_name,DOB
7499,7914,Taru Lahti-Liukkonen,Taru Lahti,1992-12-09


In [503]:
query = """
    SELECT BVB_ID, FIVB_Name
    FROM tb_df_unique
    WHERE tb_df_unique. BVB_ID not in (
        SELECT player_id
        FROM allplayers_copy)
"""

# Execute the SQL query
not_merged_players_df = ps.sqldf(query, globals())

# Display the players who didn't merge into allplayers_df
print(not_merged_players_df)

Empty DataFrame
Columns: [BVB_ID, FIVB_Name]
Index: []


In [505]:
allplayers_copy.to_csv("./data/out/allplayers_updated.csv", index=False)

so, we captured all players in our file and merged them with bigtimestat 

In [508]:
big_stat_players_df_unique = big_stat_players_df[
    ["gender", "player_name", "DOB", "height_inches"]
].drop_duplicates()
big_stat_players_df_unique.shape

(10089, 4)

In [509]:
# # # Merge big_stat_players_df with allplayers_copy
merged_players_df = pd.merge(
    allplayers_copy,
    big_stat_players_df_unique,
    left_on="big_stat_player_name",
    right_on="player_name",
    how="inner",
)

merged_players_df.rename(columns={"player_id": "BVB_ID"}, inplace=True)
merged_players_df.to_csv("./data/out/merged_players.csv", index=False)

In [510]:
merged_players_df.shape


(10021, 22)

In [511]:
## check for blackteam
#  SQL query to find players in big_stat_players_df who didn't merge into allplayers_df
query = """
    SELECT BVB_ID, FIVB_Name
    FROM tb_df_unique
    WHERE tb_df_unique.BVB_ID not in (
        SELECT BVB_ID
        FROM merged_players_df)
"""

# Execute the SQL query
not_merged_players_df = ps.sqldf(query, globals())

# Display the players who didn't merge into allplayers_df
print(not_merged_players_df)

Empty DataFrame
Columns: [BVB_ID, FIVB_Name]
Index: []


we have all black team covered 

Enriching dataset from the data from big stat volleyball

In [512]:
merged_players_df.loc[
    merged_players_df["height_inches_x"].isnull(), "height_inches_x"
] = merged_players_df["height_inches_y"]

merged_players_df.drop(columns=["height_inches_y"], inplace=True)
merged_players_df.loc[merged_players_df["DOB_x"].isnull(), "DOB_x"] = merged_players_df[
    "DOB_y"
]
merged_players_df.drop(columns=["DOB_str", "DOB_date", "DOB_y"], inplace=True)

In [514]:
# merged_players_df.loc[
#     merged_players_df["player_country_x"].isnull(), "player_country_x"
# ] = merged_players_df["player_country_y"]
# merged_players_df.drop(columns=["player_country_y"], inplace=True)


In [515]:
merged_players_df.drop(columns=["player_name_y", "height_cm"], inplace=True)

column_mapping = {
    "player_name_x": "player_name",
    "DOB_x": "DOB",
    "height_inches_x": "height_in",
    # "player_country_x": "player_country",
}

# Rename the columns by removing the "_x" suffix
merged_players_df.rename(columns=column_mapping, inplace=True)


In [516]:
# calculate podiu percentage
merged_players_df["podium%"] = (
    merged_players_df["total_points"] / merged_players_df["total_played"]
)
merged_players_df.isnull().sum()

BVB_ID                     0
player_name                0
player_country             0
DOB                     1000
home_town               3478
resides                 1928
total_first               82
total_second              82
total_third               82
total_forth               82
total_played              82
total_points              82
podium%                   82
height_in               3704
big_stat_player_name       0
gender                     0
dtype: int64

In [517]:
merged_players_df.shape


(10021, 16)

In [518]:
columns = ["BVB_ID", "isBlack_Challenge", "isBlack_Elite16"]
tb_df_tomerge = tb_df_unique[columns]

## final merge then I will need to calculate isBlack, average match time and partners
all_palyers_merged_df = pd.merge(
    merged_players_df, tb_df_tomerge, on="BVB_ID", how="left"
)

all_palyers_merged_df.shape

(10021, 18)

In [519]:
all_palyers_merged_df["isBlack_Challenge"].fillna(False, inplace=True)
# all_palyers_merged_df["isBlack_Challenge"].value_counts()

all_palyers_merged_df["isBlack_Elite16"].fillna(False, inplace=True)


all_palyers_merged_df["isBlack"] = (
    all_palyers_merged_df["isBlack_Elite16"]
    | all_palyers_merged_df["isBlack_Challenge"]
)

all_palyers_merged_df["isBlack"].value_counts()


False    9891
True      130
Name: isBlack, dtype: int64

In [520]:
# update isblack , after removing dublicates, I remved some info

all_players_merged_df = all_palyers_merged_df.merge(
    tb_df[["BVB_ID", "isBlack_Elite16"]],
    on="BVB_ID",
    how="left",
    suffixes=("_black", ""),
)

all_players_merged_df["isBlack_Elite16_black"].fillna(False, inplace=True)
all_players_merged_df["isBlack_Elite16_black"].value_counts()


False    10131
True        60
Name: isBlack_Elite16_black, dtype: int64

In [521]:
all_players_merged_df.drop(columns=["isBlack_Elite16"], inplace=True)
all_players_merged_df.rename(
    columns={"isBlack_Elite16_black": "isBlack_Elite16"}, inplace=True
)

In [419]:
## connecting with matches and calculating average match time and amount of distict  teams 
all_players_merged_df.columns


Index(['gender', 'player_name', 'DOB', 'height_in', 'player_country', 'BVB_ID',
       'home_town', 'resides', 'total_first', 'total_second', 'total_third',
       'total_forth', 'total_played', 'total_points', 'podium%',
       'big_stat_player_name', 'isBlack_Challenge', 'isBlack_Elite16',
       'isBlack'],
      dtype='object')

In [522]:
matches_df.columns
# matches_df["duration"].head()


def transform_duration(str):
    try:
        (hours, min) = str.split(":")
        duration = int(hours) * 60 + int(min)
        return duration
    except Exception:
        return math.nan


matches_df["duration_int"] = matches_df["duration"].apply(transform_duration)
# matches_df[matches_df["duration_int"] > 120][["duration", "duration_int"]]
big_stat_players_df["duration_int"] = big_stat_players_df["duration"].apply(
    transform_duration
)

C:\Users\Marina\AppData\Local\Temp\ipykernel_13004\1188183611.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_df["duration_int"] = matches_df["duration"].apply(transform_duration)


In [546]:
  matches_df.to_csv("./data/out/matches.csv", index=False)


In [475]:
# big_stat_players_df.isna().sum()

# all_players_merged_df

# big_stat_player_name
# DOB

average_duration = (
    big_stat_players_df.groupby(["player_name", "DOB"])["duration_int"]
    .mean()
    .reset_index()
)
average_duration.rename(columns={"duration_int": "duration_avg"}, inplace=True)

median_duration = (
    big_stat_players_df.groupby(["player_name", "DOB"])["duration_int"]
    .median()
    .reset_index()
)
median_duration.rename(columns={"duration_int": "duration_median"}, inplace=True)

# Calculate the number of distinct partners for each player (player_name, dob)
distinct_partners_count = (
    big_stat_players_df.groupby(["player_name", "DOB"])["partner"]
    .nunique()
    .reset_index()
)
distinct_partners_count.rename(columns={"partner": "partners_count"}, inplace=True)

# Merge 'average_duration' and 'distinct_partners_count' based on 'player_name' and 'DOB'
duration_partners_df = pd.merge(
    average_duration,
    distinct_partners_count,
    on=["player_name", "DOB"],
)

all_duration_partners_df = pd.merge(
    duration_partners_df,
    median_duration,
    on=["player_name", "DOB"],
)

all_duration_partners_df.describe()


,duration_avg,partners_count,duration_median
count,8663.000000,8986.000000,8663.000000
mean,40.173350,3.137436,39.037054
std,7.461350,3.855793,7.318043
min,11.000000,1.000000,11.000000
25%,35.333333,1.000000,34.000000
50%,40.253731,2.000000,39.000000
75%,44.682512,4.000000,43.000000
max,84.000000,61.000000,84.000000


In [477]:
# all_duration_partners_df[duration_partners_df["partners_count"] > 60]
all_duration_partners_df.columns


Index(['player_name', 'DOB', 'duration_avg', 'partners_count',
       'duration_median'],
      dtype='object')

In [479]:
all_players_merged_df.columns


Index(['gender', 'player_name', 'DOB', 'height_in', 'player_country', 'BVB_ID',
       'home_town', 'resides', 'total_first', 'total_second', 'total_third',
       'total_forth', 'total_played', 'total_points', 'podium%',
       'big_stat_player_name', 'isBlack_Challenge', 'isBlack_Elite16',
       'isBlack'],
      dtype='object')

In [539]:
all_players_merged_df.drop_duplicates(subset="BVB_ID", keep="first", inplace=True)

all_players_merged_df[all_players_merged_df.duplicated("BVB_ID", keep=False)][
    ["BVB_ID", "player_name", "big_stat_player_name"]
]

,BVB_ID,player_name,big_stat_player_name


In [540]:
## merge with all_players_merged_df.columns
# all_duration_partners_df
players_merged_df = pd.merge(
    all_players_merged_df,
    all_duration_partners_df,
    how="left",
    left_on=["big_stat_player_name", "DOB"],
    right_on=["player_name", "DOB"],
)


In [541]:
## Impute height with the average height within each 'player_country' and 'gender'
players_merged_df["height_imputed"] = players_merged_df.groupby(
    ["player_country", "gender"]
)["height_in"].transform(lambda x: x.fillna(x.mean()))

players_merged_df["height_imputed"] = players_merged_df.groupby(["gender"])[
    "height_in"
].transform(lambda x: x.fillna(x.mean()))

players_merged_df[["height_imputed", "height_in"]]


,height_imputed,height_in
0,75.096019,NaN
1,73.000000,73.0
2,76.000000,76.0
3,74.000000,74.0
4,77.000000,77.0
...,...,...
9990,75.096019,NaN
9991,75.096019,NaN
9992,75.096019,NaN
9993,75.096019,NaN


In [529]:
players_merged_df.columns

Index(['BVB_ID', 'player_name_x', 'player_country', 'DOB', 'home_town',
       'resides', 'total_first', 'total_second', 'total_third', 'total_forth',
       'total_played', 'total_points', 'podium%', 'height_in',
       'big_stat_player_name', 'gender', 'isBlack_Challenge',
       'isBlack_Elite16', 'isBlack', 'duration_avg', 'partners_count',
       'duration_median', 'height_imputed'],
      dtype='object')

In [542]:
players_merged_df.rename(columns={"player_name_x": "player_name"}, inplace=True)


In [543]:
players_merged_df[players_merged_df.duplicated("BVB_ID", keep=False)][
    ["BVB_ID", "player_name", "big_stat_player_name"]
]


,BVB_ID,player_name,big_stat_player_name


In [544]:

players_merged_df["podium%"] = (
    players_merged_df["total_first"] + players_merged_df["total_second"] + players_merged_df["total_third"]
) / players_merged_df["total_played"]
players_merged_df["podium%"] = players_merged_df["podium%"].round(2)

players_merged_df.to_csv("./data/out/players1.csv", index=False)
